# Data Preprocessing

This script contains our standard preprocessing model for all CBT runs, which includes:
* 4mm smoothing
* 80s High pass Filter cutoff
* slice timing correction (interleaved up)
* registration: 6 DOF
* normalization: 12 DOF nonlinear (10 mm warp sphere)

In [1]:
import sys
import os
import numpy as np

In [ ]:
def standard(SUBJ, run_name, EXP_DIR):

    run_type = run_name[:-1]
    run_num = run_name[-1:]

    # specify the number of timepoints
    if run_name == 'LOCALIZER':
        n_tmpts = 400
    else:
        n_tmpts = 473

    # calculate total datapoints
    n_total_datapts = n_tmpts * 64 * 64 * 18


    # config vars:
    template = 'CBT_preprocessing_standard.fsf'
    template_path = os.path.join(EXP_DIR, 'scripts/analysis_scripts/fMRI_workflow/FEAT_templates/preprocessing', template)
    anat_dir = os.path.join(EXP_DIR, 'data/anat', SUBJ)
    
    # inputs
    input_data = os.path.join(EXP_DIR,'data/func', SUBJ, (run_name + '.nii.gz'))
    #full_anat = os.path.join(anat_dir, 'full_anat_brain.nii.gz')
    full_anat = os.path.join(anat_dir, 'full_anat_skull_and_brain.nii.gz')
    
    # outputs
    output_dir = os.path.join(EXP_DIR, 'analysis/preprocessing', SUBJ)
    if not os.path.isdir(output_dir):
        os.makedirs(output_dir)
    output_FSL = os.path.join(output_dir, run_name)
    
    # open the template file in 'read' mode, extract all of the text
    template_file = open(template_path, 'r')
    text = template_file.read() 
    template_file.close()
    
    # make substitutions to template text
    text = text.replace('SUB_OUTPUT_DIR_SUB', output_FSL)
    text = text.replace('SUB_INPUT_DATA_SUB', input_data)
    text = text.replace('SUB_FULL_ANAT_SUB', full_anat)
    text = text.replace('SUB_TMPTS_SUB', str(n_tmpts))
    text = text.replace('SUB_TOTAL_DATAPTS_SUB', str(n_total_datapts))
    
    # write the new text to a unique .fsf file
    new_feat_fname = os.path.join(output_dir, (run_name + '_preprocessing.fsf'))
    new_feat = open(new_feat_fname, 'w')
    new_feat.write(text)
    new_feat.close()
    
    # create job
    job = ('feat ' + new_feat_fname)
    return job